In [2]:
import pandas as pd
import numpy as np
import datetime
import sys
import MySQLdb
import matplotlib.pyplot as plt 
import matplotlib.dates as dates
import seaborn as sns
from IPython.display import display 
from ipywidgets import interact, fixed, widgets, HBox
from os.path import exists 

%matplotlib inline



In [3]:
SERVER_IP = "10.10.0.81"
TICK_INIT = 10*60*60
RESAMP_INIT = 120
MAX_DF_LEN = 1
DELTA_SAMP_MIN = 200
RESAMP_MAX = 120000*30


someCols = np.sort(['date', 'atc1_mean', 'atc2_mean', 'atc3_mean', 
                    'atc4_mean', 'atc5_mean', 'atc6_mean',
                    'roomtc1_mean', 'roomtc2_mean', 'fridgec_rms', 'fridgev_rms', 
                    'rtc1a_mean', 'rtc2a_mean', 'rtc3a_mean', 'rtc4a_mean',
                    'rtc5a_mean', 'rtc6a_mean', 'rtc7a_mean', 'rtc8a_mean',
                    'pressa_mean', 'htrva_rms', 'htria_rms', 'pidseta', 
                    'pidtca', 'fpidset', 'fpidtc',  
                    'rtc1b_mean', 'rtc2b_mean', 'rtc3b_mean', 'rtc4b_mean',
                    'rtc5b_mean', 'rtc6b_mean', 'rtc7b_mean', 'rtc8b_mean',
                    'pressb_mean', 'htrib_rms', 'htrvb_rms', 'pidsetb', 'pidtcb', 
                    'coilv_rms', 'coilcsv_rms', 'coilenergy'])


In [4]:
def printnow(val):
    print(val)
    sys.stdout.flush()

def connectDbase(dbase):
    conn = MySQLdb.connect(host=SERVER_IP, user="ihs01", passwd="!ndustr!al13", db=dbase)
    cursor = conn.cursor()
    return cursor

def changeDbase(dbase, cursor):
    cursor.execute('use %s;'%dbase) 

def readQuery(query, cursor):
    cursor.execute(query)
    data = cursor.fetchall()
    return data

def firstTime(cursor, tableName):
    query = 'select date from %s order by date limit 1'%tableName
    cursor.execute(query)
    data = cursor.fetchall()
    data = data[0][0]
    sdata = data.strftime("%Y-%m-%d %H:%M:%S")
    return sdata

def lastTime(cursor, tableName):
    query = 'select date from %s order by date desc limit 1'%tableName
    cursor.execute(query)
    data = cursor.fetchall()
    data = data[0][0]
    sdata = data.strftime("%Y-%m-%d %H:%M:%S")
    return sdata

def getColName(cursor, tableName):
    cursor.execute('desc %s'%tableName)
    colName = [column[0] for column in cursor.fetchall()]
    colDict ={}
    for i,j in enumerate(colName):
        colDict.update({i:j})
    return colDict

def lastTimeDF(df):
    return df.index[-1].strftime("%Y-%m-%d %H:%M:%S") 

def firstTimeDF(df):
    return df.index[0].strftime("%Y-%m-%d %H:%M:%S") 


In [5]:
def initDataFrame(data, colName):
    if len(data) == 0:
        df = pd.DataFrame(columns=colName)
    else:
        # create data frame with headers and date for index
        df = pd.DataFrame([[ij for ij in i] for i in data])
        df.rename(columns=colName, inplace=True);
        df.set_index(pd.to_datetime(df['date'].values), inplace=True)
    return df    

def readNoDateTable(cursor, tableName, query = 'SELECT * FROM %s'):
    # get col headers into a dictionary from mysql database
    colName = getColName(cursor, tableName)
    # execute query   
    cursor.execute(query%tableName)
    data = cursor.fetchall()
    df = pd.DataFrame([[ij for ij in i] for i in data])
    df.rename(columns=colName, inplace=True);
    return df

def readTable(cursor, tableName, query = 'SELECT * FROM %s'):
    # get col headers into a dictionary from mysql database
    colName = getColName(cursor, tableName)
    # execute query   
    cursor.execute(query%tableName)
    data = cursor.fetchall()
    df = initDataFrame(data, colName)
    return df

def readDateTable(cursor, tableName, start='2015-12-12 11:10:23', end='2015-12-12 13:10:23'):
    # get col headers into a dictionary from mysql database
    colName = getColName(cursor, tableName)
    #print 'SELECT * FROM %s WHERE date BETWEEN \'%s\' and \'%s\''%(tableName,start,end)
    cursor.execute('SELECT * FROM %s WHERE date BETWEEN \'%s\' and \'%s\''%(tableName,start,end))
    data = cursor.fetchall()
    df = initDataFrame(data, colName)
    return df

def readPartTable(cursor, tableName, start='2015-12-12 11:10:23', end='2015-12-12 13:10:23'):
    global someCols
    
    qString = ''
    for i in someCols:
        qString = qString + ('%s,'%(i)) 
    qString = qString[0:-1]    
    colName = dict(zip(np.arange(len(someCols)), someCols))
    
    #print 'SELECT %s FROM %s WHERE date BETWEEN \'%s\' and \'%s\''%(qString,tableName,start,end)
    cursor.execute('SELECT %s FROM %s WHERE date BETWEEN \'%s\' and \'%s\''%(qString,tableName,start,end))
    data = cursor.fetchall()
    df = initDataFrame(data, colName)
    return df

def rmsReadTable(cursor, tableName, blockLenInSec, start='2015-12-12 11:10:23', end='2015-12-12 11:20:23'):    
    colName = getColName(cursor, tableName)
    query = '''
    SELECT date, SQRT(SUM(POW(htrv,2))/COUNT(htrv)), 
           SQRT(SUM(POW(htri,2))/COUNT(htri)), SQRT(SUM(POW(fridgev,2))/COUNT(fridgev)), 
           SQRT(SUM(POW(fridgei,2))/COUNT(fridgei))
    FROM %s 
    WHERE date between \'%s\' AND \'%s\'
    GROUP BY unix_timestamp(date) div %d
    '''%(tableName, start, end, blockLenInSec)
    out = readQuery(query)
    df = initDataFrame(out, colName)
    return df

def getNextRMSTimeBlock(cursor, df, tablename, blockLenInSec):
    if df.empty:
        print 'Null DataFrame'
        new = pd.DataFrame
    else:
        start = lastTimeDF(df)
        end = lastTime(cursor, tablename)
        new  = rmsReadTable(cursor, tablename, 120, start, end)
        if len(new) > 0:
            # get rid of first row
            new.drop(new.index[0], inplace=True)
    return new

def getNextTimeBlock(cursor, df, tablename):
    if df.empty:
        print 'Null DataFrame'
        new = pd.DataFrame
    else:
        start = lastTimeDF(df)
        end = lastTime(cursor, tablename)
        new  = readPartTable(cursor, tablename, start, end)
        if len(new) > 0:
            # get rid of first row
            new.drop(new.index[0], inplace=True)
            df = df.append(new)
    return df

def getOneRMSBlock(cursor, tablenName, start, end):
    top = '''
           select  *
           from    %s
           '''
    bot = '''where   date between \'%s\' and \'%s\' '''%(start.strftime("%Y-%m-%d %H:%M:%S"),
                                                        end.strftime("%Y-%m-%d %H:%M:%S"))
    query = top + bot
    df = readTable(tableName, query)
    df.drop(['seconds', 'date'], axis=1, inplace=True)
    N = len(df)
    df = df **2    
    out = np.sqrt(df.sum() / float(N))
    out = out.to_frame().transpose()
    df.ix[0,:] = out.values    
    return df

def readLastNTable(cursor, tableName, N=1):
    
    # get col headers into a dictionary from mysql database
    colName = getColName(cursor, tableName)
    #print 'SELECT * FROM %s WHERE date BETWEEN \'%s\' and \'%s\''%(tableName,start,end)
    cursor.execute('SELECT * FROM %s ORDER BY DATE DESC LIMIT %d'%(tableName, N))
    data = cursor.fetchall()
    df = initDataFrame(data, colName)
    return df


In [6]:

def update(cursor,df1, tableName, fileName):
    global NEW_TABLE
    global someCols
    df1 = df1[someCols]
    df1 = getNextTimeBlock(cursor, df1, tableName)
    if NEW_TABLE >=7:
        df1 = calcPower(df1)
    
    print "Updated Data"
    print 'Last Database Entry:  %s'%lastTime(cursor, tableName)
    print 'Last Dataframe Entry: %s'%lastTimeDF(df1)
    pickleDataFrame(df1, fileName)    
    
    return df1

def pickleDataFrame(df, name):
    df.to_pickle(name)
    return

def readDataFrame(name):
    df = pd.read_pickle(name)
    return df



In [7]:
tableWid = widgets.Dropdown(
    description='Run/Table',
    options={'Table 005' : 5,
             'Table 007' : 7,
             'Table 006' : 6,
             'Table 008' : 8,
             'Table 009' : 9,
             'Table 010' : 10,
             'Table 011' : 11,
             'Table 012' : 12},
    value=11,
)


reactorWid = widgets.Dropdown(
    description='Reactor/dBase',
    options={'Reactor 005' : 5,
             'Reactor 006' : 6,
             'Reactor 007' : 7},
    value=7,
)


updateWid = widgets.Checkbox(
description='Update Data',
value=False
)
    
# table = tableWid
# dBase = reactorWid
# new = updateWid
# ChooseBox = HBox([dBase, table, new])
# df = pd.DataFrame()
# OLD_TABLE = 0
# NEW_TABLE = 0

# def initConsole(tbl, rct, getData, df):
#     tbl = tbl.value
#     rct = rct.value
#     getData = getData.value
#     global OLD_TABLE
#     global NEW_TABLE
    
#     NEW_TABLE = tbl
#     TABLE = '{0:03d}'.format(tbl)
#     RCT = '{0:03d}'.format(rct)
    
#     pickleFile = '../DATA/rct' + RCT + 'run' + TABLE + '.pkl'
#     if rct==5:
#         dbName = 'rct' + RCT[1:] + 'db'
#         tbName = TABLE + '_data'
#     else:
#         dbName = 'rct' + RCT 
#         tbName = 'test_plcdata'

#     if NEW_TABLE!=OLD_TABLE:
#         # if file exist return it or else do a dbase read
#         OLD_TABLE = tbl
#         if exists(pickleFile):
#             df = readDataFrame(pickleFile)
#         else:
#             default_start = '1999-12-28 18:00:00'
#             time_now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#             cursor = connectDbase(dbName)
#             df = readPartTable(cursor,tbName, default_start, time_now)
    
#     elif getData:
#         if df.empty:
#             print 'Empty DataFrame'
#         else:
#             cursor = connectDbase(dbName)
#             df = update(cursor, df, tbName, pickleFile)    
            
#     return df